# MongoDB insert all jurybook

In [142]:
import re
import _uniout
import pymongo
from pymongo import MongoClient
#make a connection to mongodb 
client = pymongo.MongoClient('localhost',27017)
#assign db_name to db
db = client.test
#make the collection in the db of test
samples = db['samples']
#print the case number in db.dday
content = u''
for dic in db.samples.find():
    #print _uniout.unescape(str(i),'utf-8') 
    content += dic['content'] #append all the jurybook to the content
    

# Cut the word by all apending jurybook

# Please put the "stopwords.txt" with the same directory of ipython

In [169]:
#-*- utf-8 -*-
import codecs
import operator
import os

cutlist = " <>/:：;；,、＂’，.。！？｢\"\'\\\n\r《》“”!@#$%^&*()".decode("utf-8")  ##列出標點符號，並轉換成utf-8的格式
def cutSentence(content, stopwords): ##放入原始文章路徑, 增加斷詞的list
    sentence = ""
    textList = []
       
    for line in content:
        line = line.strip() ##清除空白
        #line = line.replace(' ', '')
        for word in line:
            if word not in cutlist: #如果文字不是標點符號，就把字加到句子中
                sentence += word
                #print sentence
            else:
                #stopwords =[u'裁判']
                for stopword in stopwords:  #清除關鍵字
                    #print stopword
                    sentence = sentence.replace(stopword,'')
                    #print type(stopword)
                #print sentence
                textList.append(sentence) #把文章句子依標點符號與空白隔開
                sentence = ""
            #for ele in textList: #顯示文章所斷句子
                #print ele
    return textList
    
def ngram(textLists,n,minFreq): 
    #首參數放處理好的文章(LIST檔，utf-8編碼)次參數放字詞的長度單位，第三個參數放多少頻次以上
    words=[]     #存放擷取出來的字詞
    words_freq={}#存放字詞:計算個數 
    result= []
    for textList in textLists:
        for w in range(len(textList)-(n-1)): #要讀取的長度隨字詞長度改變
            words.append(textList[w:w+n])    #抓取長度w-(n-1)的字串

    for word in words:
        if word not in words_freq:               #如果這個字詞還沒有被放在字典檔中
            words_freq[word] = words.count(word) #就開一個新的字詞，裡面放入字詞計算的頻次
 
    words_freq = sorted(words_freq.iteritems(),key=operator.itemgetter(1),reverse=True) #change words_freq from dict to list 
    
    for word in words_freq:
        if word[1] >= minFreq:
            result.append(word)
    return result ##回傳一個陣列[(詞,頻次),(詞,頻次)]

def longTermPriority(path, maxTermLength, minFreq):   #目前設定只傳回長詞longTerms
    stopwords=[]
    with codecs.open('./stopwords.txt','r' ,'utf-8') as f :
        readstopwords = f.readlines() #從txt檔讀取每行的停止詞
    for ele in readstopwords:
        stopwords.append(ele.strip())#從每個停止詞中，去除該死的換行符號
    longTermsFreq=[]      #長詞+次數分配
    for i in range(maxTermLength,1,-1): ##字詞數由大至小
        text_list = cutSentence(path,stopwords)  
        words_freq = ngram(text_list,i, minFreq) 
        for word_freq in words_freq:
            longTermsFreq.append(word_freq) #將長詞和次數加入另外一個list  分成兩個檔儲存的用意是減少迴圈次數
    return longTermsFreq

longTermFreq = longTermPriority(content,4,2) ##最長詞5個字、出現頻次1次以上
#print longTermFreq
#for i in longTermFreq:
 #   print i[0],i[1]   #印出斷字結果


# 切字結果轉存至記事本

In [170]:
import _uniout
import codecs,os
try:
    os.remove('ngramwords.txt')
except:
    pass
for i in longTermFreq:
    #tt = str(i[1]).decode('utf8')
    #print type(tt)
    f = codecs.open('ngramwords.txt','a+','utf-8')
    llist = i[0]+','+str(i[1]).decode('utf8')+'\n'   #印出斷字結果
    f.write(llist)
    f.close()